In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

train=pd.read_csv("engtamilTrain.csv")
train=train.drop(["Unnamed: 0"],axis=1)
english_sentences=train["en"]
tamil_sentence=train['ta']
english_sentences=english_sentences.head(1000)
tamil_sentences=tamil_sentence.head(1000)

#### #Function to add SOS and EOS to the statement


In [2]:
#modify each statement in a list by adding specific tokens. 

def addSosEos(seriesSentence):
    # Define the <SOS> and <EOS> tokens
    sos_token = "<SOS>"
    eos_token = "<EOS>"

    # Add <SOS> and <EOS> tokens to each statement
    statements_with_tokens = [f"{sos_token} {statement} {eos_token}" for statement in seriesSentence]

    english_sent=[]
    # Print the statements with tokens
    for statement in statements_with_tokens:
        english_sent.append(statement)
        print(statement)
    return english_sent


In [3]:
#The function adds start-of-sequence (<SOS>) and end-of-sequence (<EOS>) tokens to each sentence in both lists.

english_sent_SE=addSosEos(english_sentences)
tamil_sent_SE=addSosEos(tamil_sentences)

<SOS> MMA vice president Qazi Hussain Ahmad declared last month: 'We are not extremists.
 <EOS>
<SOS> Information has surfaced in recent years suggesting that Julius Rosenberg was involved in passing some form of intelligence to Soviet officials during the Second World War.
 <EOS>
<SOS> And Azor begat Sadoc; and Sadoc begat Achim; and Achim begat Eliud;
 <EOS>
<SOS> She says she knows what is going on, but can do nothing about it.
 <EOS>
<SOS> And be it indeed that I have erred, my error remains with myself.
 <EOS>
<SOS> Finally, the columnist fails to tell us who among the political leaders of the bourgeoisie, past and present, he counts among the paragons of morality.
 <EOS>
<SOS> These include the British Tamil Forum, La Maison du Tamil Eelam (France), the Canadian Tamil Congress, and the Swiss Tamil Forum.
 <EOS>
<SOS> Vijay accompanied with his wife and daughter enjoyed the film 'Anjathey'.
 <EOS>
<SOS> Both Musharraf and Vajpayee have exploited the current war drive to divert pub

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#### filters="", you are indicating that no filtering should be applied. This means that all characters, including punctuation and special symbols, will be retained in the texts.
Example
#### Vocabulary Size (English): 9     
Word Index (English): {'hello,': 1, 'world!': 2, 'this': 3, 'is': 4, 'a': 5, 'test.': 6, 'how': 7, 'are': 8, 'you?': 9}
Sequences (English): [[1, 2], [3, 4, 5, 6], [7, 8, 9]]

####  all unique words and the necessary padding index, allowing for proper input into models like neural networks.

In [5]:
tamil_tokenizer = Tokenizer(filters="")
tamil_tokenizer.fit_on_texts(tamil_sent_SE)
tamil_vocab_size = len(tamil_tokenizer.word_index) + 1
tamil_sequences = tamil_tokenizer.texts_to_sequences(tamil_sent_SE)

In [6]:
tamil_tokenizer

In [7]:
tamil_vocab_size

9922

In [8]:
tamil_sequences
#Each sequence begins with 1<sos>   end 2(eos)

[[1, 1836, 44, 1837, 1838, 1839, 1840, 148, 78, 884, 1841, 1842, 885, 282, 2],
 [1,
  1843,
  69,
  886,
  1844,
  1845,
  3,
  1846,
  1847,
  283,
  551,
  1848,
  1849,
  1850,
  1851,
  1852,
  1853,
  2],
 [1, 1854, 1855, 887, 1856, 1857, 887, 1858, 1859, 1860, 2],
 [1, 284, 1861, 33, 888, 1862, 35, 149, 1863, 382, 1864, 35, 552, 889, 2],
 [1, 11, 1865, 1866, 17, 1867, 1868, 1869, 2],
 [1, 1870, 1871, 1872, 1873, 1874, 553, 1875, 2],
 [1,
  6,
  1876,
  1877,
  383,
  1878,
  1879,
  1880,
  1881,
  383,
  1882,
  1883,
  233,
  1884,
  383,
  1885,
  1886,
  383,
  1887,
  1888,
  384,
  890,
  2],
 [1, 26, 1889, 1890, 1891, 111, 1892, 1893, 2],
 [1,
  1894,
  4,
  1895,
  1896,
  554,
  150,
  891,
  385,
  91,
  285,
  1897,
  1898,
  554,
  1899,
  1900,
  1901,
  1902,
  286,
  555,
  1903,
  892,
  2],
 [1, 36, 284, 1904, 2],
 [1,
  1905,
  234,
  19,
  893,
  1906,
  127,
  92,
  894,
  556,
  1907,
  5,
  1908,
  12,
  3,
  895,
  287,
  1909,
  896,
  79,
  2],
 [1,
  80,

In [9]:
# Tokenize the English and Tamil sentences
english_tokenizer = Tokenizer(filters="")
english_tokenizer.fit_on_texts(english_sent_SE)
english_vocab_size = len(english_tokenizer.word_index) + 1
english_sequences = english_tokenizer.texts_to_sequences(english_sent_SE)

In [10]:
max_input_seq_length=20
max_output_seq_length=20

####  particularly neural networks, require inputs to have a consistent shape. Padding ensures that all sequences are of the same length, 
only depending words take integers remain 0

In [11]:
# Pad sequences to a fixed length
input_sequences = pad_sequences(english_sequences, maxlen=max_input_seq_length, padding='post')
output_sequences = pad_sequences(tamil_sequences, maxlen=max_output_seq_length, padding='post')

In [12]:
input_sequences

array([[   2, 2036, 2037, ...,    0,    0,    0],
       [   9, 2044, 2045, ...,  105,  342,    3],
       [   2,    5, 2047, ...,    0,    0,    0],
       ...,
       [6958, 6959,   68, ...,   16, 6963,    3],
       [1449,  207, 6967, ..., 6975, 6976,    3],
       [   2,   45,   85, ...,    0,    0,    0]])

In [13]:
output_sequences

array([[   1, 1836,   44, ...,    0,    0,    0],
       [   1, 1843,   69, ...,    2,    0,    0],
       [   1, 1854, 1855, ...,    0,    0,    0],
       ...,
       [9890, 9891,   86, ..., 9899, 9900,    2],
       [9904, 9905, 9906, ..., 1036, 9919,    2],
       [   1,  195,   25, ...,    0,    0,    0]])

In [14]:
output_sequences[0]

array([   1, 1836,   44, 1837, 1838, 1839, 1840,  148,   78,  884, 1841,
       1842,  885,  282,    2,    0,    0,    0,    0,    0])

In [15]:
output_sequences[1]

array([   1, 1843,   69,  886, 1844, 1845,    3, 1846, 1847,  283,  551,
       1848, 1849, 1850, 1851, 1852, 1853,    2,    0,    0])

##### The presence of 0s at the end of some sequences indicates padding. Padding is used to ensure that all sequences in a batch have the same length, which is essential for feeding the data into neural networks.

#### Teacher forcing is a training strategy where the true output from the previous time step is used as the input for the next time step in the decoder, rather than using the decoder's own predictions.

#### For example, if the output sequence is [<SOS>, word1, word2, word3, <EOS>], the decoder input sequence for teacher forcing would be [<SOS>, word1, word2, word3].

In [16]:
# Prepare the decoder input and output sequences for teacher forcing
decoder_input_sequences = np.zeros_like(output_sequences)


In [17]:
decoder_input_sequences

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [18]:
output_sequences

array([[   1, 1836,   44, ...,    0,    0,    0],
       [   1, 1843,   69, ...,    2,    0,    0],
       [   1, 1854, 1855, ...,    0,    0,    0],
       ...,
       [9890, 9891,   86, ..., 9899, 9900,    2],
       [9904, 9905, 9906, ..., 1036, 9919,    2],
       [   1,  195,   25, ...,    0,    0,    0]])

In [19]:
output_sequences[0]

array([   1, 1836,   44, 1837, 1838, 1839, 1840,  148,   78,  884, 1841,
       1842,  885,  282,    2,    0,    0,    0,    0,    0])

####[:, :-1]This expression selects all rows of the array while including all columns except the last one. 

In [20]:
output_sequences[:, :-1][0]

array([   1, 1836,   44, 1837, 1838, 1839, 1840,  148,   78,  884, 1841,
       1842,  885,  282,    2,    0,    0,    0,    0])

In [21]:
decoder_input_sequences[:, 1:] = output_sequences[:, :-1]

In [22]:
decoder_input_sequences[:, 1:]

array([[   1, 1836,   44, ...,    0,    0,    0],
       [   1, 1843,   69, ..., 1853,    2,    0],
       [   1, 1854, 1855, ...,    0,    0,    0],
       ...,
       [9890, 9891,   86, ...,    3, 9899, 9900],
       [9904, 9905, 9906, ...,  232, 1036, 9919],
       [   1,  195,   25, ...,    0,    0,    0]])

In [23]:
decoder_input_sequences[:, 1:][0]

array([   1, 1836,   44, 1837, 1838, 1839, 1840,  148,   78,  884, 1841,
       1842,  885,  282,    2,    0,    0,    0,    0])

#### decoder_input_sequences[:, 0] is a way to access the first column of the decoder input sequences, which typically contains the start-of-sequence (SOS) tokens. This is a critical step in preparing the input for the decoder in a sequence-to-sequence model, as it indicates where the output sequence generation begins. This setup is essential for tasks such as machine translation, text summarization, 

In [24]:
decoder_input_sequences[:, 0] = tamil_tokenizer.word_index['<sos>']


In [25]:
decoder_input_sequences[:, 0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [26]:
decoder_input_sequences[:, 0][0]

1

In [27]:
tamil_tokenizer.word_index['<sos>']

1

#### During training, the decoder often starts with the SOS token, allowing it to generate subsequent tokens based on the context provided by previous tokens.
By providing the SOS token as the initial input, the decoder can learn to generate coherent output sequences.

In [28]:
decoder_output_sequences = np.eye(tamil_vocab_size)[output_sequences]

In [35]:
output_sequences

array([[   1, 1836,   44, ...,    0,    0,    0],
       [   1, 1843,   69, ...,    2,    0,    0],
       [   1, 1854, 1855, ...,    0,    0,    0],
       ...,
       [9890, 9891,   86, ..., 9899, 9900,    2],
       [9904, 9905, 9906, ..., 1036, 9919,    2],
       [   1,  195,   25, ...,    0,    0,    0]])

In [30]:
tamil_vocab_size

9922

In [32]:
np.eye(tamil_vocab_size)
#This creates a 9922 x 9922 identity matrix, where the diagonal elements are 1s and all other elements are 0s. 
#This matrix serves as a basis for creating one-hot encoded vectors.
#This results in a matrix of shape (9922, 9922) where:identity_matrix[i][j] = 1 if i == j (diagonal elements)identity_matrix[i][j] = 0 otherwise

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [34]:
np.eye(tamil_vocab_size)[output_sequences]

array([[[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0.

#### Resulting One-Hot Encoded Array
The result of this indexing operation is a 3D array where:
Each row corresponds to a token in the original sequences.
Each token is represented as a one-hot encoded vector.
the output might look like this:
array([[1., 0., 0., ..., 0., 0., 0.],  # One-hot for token 1

       [0., 1., 0., ..., 0., 0., 0.],  # One-hot for token 1836
       
       [0., 0., 1., ..., 0., 0., 0.],  # One-hot for token 44
       
       ...,
       [0., 0., 0., ..., 1., 0., 0.],  # One-hot for the last token in the sequence
       
       [0., 0., 0., ..., 0., 1., 0.],  # One-hot for another token
       
       [0., 0., 0., ..., 0., 0., 1.]]) # One-hot for the last token

##### By loading the model, you can leverage the learned word embeddings to perform various operations, such as obtaining word vectors and finding similar words. 

In [37]:
from gensim.models import Word2Vec

eng_model = Word2Vec.load('engmodel.bin')
tam_model = Word2Vec.load('tammodel.bin')
#The engmodel.bin file contains word embeddings that represent words in a continuous vector space. These embeddings capture semantic relationships, 
#meaning that words with similar meanings will have similar vector representations.

In [38]:
eng_model

In [40]:
 Word2Vec.load('engmodel.bin')

#### Purpose: The primary function of this line is to initialize an embedding matrix filled with zeros. This matrix will be used to store the vector representations (embeddings) of words from a vocabulary.
Dimensions:
vocab_size: This represents the number of unique words (or tokens) in your vocabulary. Each unique word will have a corresponding row in the embedding matrix.
word2vec_model.vector_size: This indicates the dimensionality of the word vectors produced by the Word2Vec model. Each word will be represented as a vector with this many features.

In [43]:
def create_embedding_matrix(word2vec_model, tokenizer, vocab_size):
    embedding_matrix = np.zeros((vocab_size, word2vec_model.vector_size))
    for word, i in tokenizer.word_index.items():
        try:
            embedding_vector = word2vec_model.wv[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            pass  # Words not found in the embedding index will be all zeros
    return embedding_matrix

eng_embedding_matrix = create_embedding_matrix(eng_model, english_tokenizer, english_vocab_size)
tam_embedding_matrix = create_embedding_matrix(tam_model, tamil_tokenizer, tamil_vocab_size)


In [44]:
eng_embedding_matrix.shape

#for one word huge dimension(100)  present
    # ex king→[0.25,−0.1,0.3,…,0.15](100 values)

(6979, 100)

In [45]:
tam_embedding_matrix.shape

(9922, 100)

In [48]:
def create_seq2seq_model(input_vocab_size, output_vocab_size, input_seq_length, output_seq_length, hidden_units, eng_embedding_matrix, tam_embedding_matrix):
    # Encoder
    encoder_inputs = Input(shape=(input_seq_length,))
    encoder_embedding = Embedding(input_vocab_size, hidden_units, weights=[eng_embedding_matrix], trainable=False)(encoder_inputs)
    encoder_lstm, encoder_state_h, encoder_state_c = LSTM(hidden_units, return_state=True)(encoder_embedding)

    # Decoder
    decoder_inputs = Input(shape=(output_seq_length,))
    decoder_embedding = Embedding(output_vocab_size, hidden_units, weights=[tam_embedding_matrix], trainable=False)(decoder_inputs)
    decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[encoder_state_h, encoder_state_c])
    decoder_dense = Dense(output_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    return model


In [49]:
# Convert target_sequences to one-hot encoded format
target_sequences = tf.keras.utils.to_categorical(output_sequences, num_classes=tamil_vocab_size)


In [50]:
model = create_seq2seq_model(english_vocab_size, tamil_vocab_size, max_input_seq_length, max_output_seq_length, 100, eng_embedding_matrix, tam_embedding_matrix)

In [51]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [52]:
# Fit the model to the data
batch_size = 32
epochs = 500
model.fit([input_sequences, output_sequences], decoder_output_sequences, batch_size=batch_size, epochs=epochs, validation_split=0.2)


Epoch 1/500
25/25 [==============================] - 17s 354ms/step - loss: 8.9562 - accuracy: 0.2575 - val_loss: 8.3715 - val_accuracy: 0.2722
Epoch 2/500
25/25 [==============================] - 5s 187ms/step - loss: 7.4785 - accuracy: 0.2684 - val_loss: 7.4978 - val_accuracy: 0.2690
Epoch 3/500
25/25 [==============================] - 5s 213ms/step - loss: 6.5464 - accuracy: 0.2686 - val_loss: 7.4496 - val_accuracy: 0.2718
Epoch 4/500
25/25 [==============================] - 5s 206ms/step - loss: 6.3748 - accuracy: 0.2692 - val_loss: 7.4964 - val_accuracy: 0.2720
Epoch 5/500
25/25 [==============================] - 6s 227ms/step - loss: 6.2988 - accuracy: 0.2693 - val_loss: 7.5522 - val_accuracy: 0.2722
Epoch 6/500
25/25 [==============================] - 5s 204ms/step - loss: 6.2467 - accuracy: 0.2691 - val_loss: 7.5169 - val_accuracy: 0.2713
Epoch 7/500
25/25 [==============================] - 5s 211ms/step - loss: 6.1792 - accuracy: 0.2691 - val_loss: 7.5471 - val_accuracy: 0.272

In [53]:
# Preprocessing the input
input_sentence = "<sos>Finally, the columnist fails to tell us who among the political leaders of the bourgeoisie, past and present, he counts among the paragons of morality<eos>"

# Convert the input sentence to sequence
input_sequence = english_tokenizer.texts_to_sequences([input_sentence])

# Pad the statement to the maximum input sequence length
input_sequence = pad_sequences(input_sequence, maxlen=max_input_seq_length, padding='post')

# Generate predictions
predictions = model.predict([input_sequence, np.zeros((1, max_output_seq_length))])

# Convert predictions to tokens
predicted_tokens = np.argmax(predictions, axis=-1)[0]

# Create index to word mapping for Tamil vocabulary
tamil_index_word = {i: w for w, i in tamil_tokenizer.word_index.items()}


# Convert tokens to text
decoded_sentence = []
for token in predicted_tokens:
    if token == 0:  # Assuming 0 is the padding token
        continue
    word = tamil_index_word.get(token)
    if word == '<eos>':
        break
    if word is not None:
        decoded_sentence.append(word)
    else:
        decoded_sentence.append('<unk>')

# Join the words to form the decoded statement
decoded_statement = ' '.join(decoded_sentence)

# Print the decoded statement
print(decoded_statement)


1/1 [==============================] - 3s 3s/step
பயங்கரமான உல் இஸ்ரவேல் வம்சத்தாரில் "சீன சாதிக்கும் பாதுகாப்பிற்கு வியட்நாம் வளையங்களையும் தரியு, பெர்சியாவின்
